In [1]:
import requests

In [281]:
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import datetime

import logging

def get_historic_data(date: datetime.datetime):
    
    date_str = date.strftime('%Y-%m-%d')
    lookup_URL = "https://www.wunderground.com/history/daily/us/wy/jackson/KJAC/date/{0}".format(date_str)
    options = webdriver.ChromeOptions();
    options.add_argument('headless'); # to run chrome in the backbroung
    driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver', options=options)  # Optional argument, if not specified will search path.
    xpath = "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr[*]"
    driver.get(lookup_URL)
    rows = WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.XPATH, xpath)))
    raw_data = []
    for row in rows:
        cols=row.find_elements_by_tag_name("td")
        data_dict = {"Time":cols[0].text, 
                     "Temperature":cols[1].text, 
                     "Dew Point":cols[2].text,
                     "Humidity":cols[3].text,
                     "Wind":cols[4].text,
                     "Wind Speed":cols[5].text, 
                     "Wind Gust":cols[6].text,
                     "Pressure":cols[7].text, 
                     "Precip.":cols[8].text,
                     "Condition":cols[9].text
                    }
        raw_data.append(data_dict)
    
    df = pd.DataFrame.from_records(raw_data)
    return df

In [282]:
import datetime

dt = datetime.datetime(2021, 1, 1)
end = datetime.datetime.now()
step = datetime.timedelta(days = 1)

dates = []

while dt < end:
    dates.append(dt)
    dt += step

In [153]:
df = pd.DataFrame()
for date in dates[:-1]:
    print("checking date"+date.strftime("%m/%d/%Y, %H:%M:%S"))
    df_temp = get_historic_data(date)
    df_temp["Time"] = pd.to_datetime(df_temp['Time']).apply(lambda x: x.replace(year=date.year, month=date.month, day=date.day))

    df = pd.concat([df, df_temp], ignore_index=True)
df

checking date01/01/2021, 00:00:00
checking date01/02/2021, 00:00:00
checking date01/03/2021, 00:00:00
checking date01/04/2021, 00:00:00
checking date01/05/2021, 00:00:00
checking date01/06/2021, 00:00:00
checking date01/07/2021, 00:00:00
checking date01/08/2021, 00:00:00
checking date01/09/2021, 00:00:00
checking date01/10/2021, 00:00:00
checking date01/11/2021, 00:00:00
checking date01/12/2021, 00:00:00


,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2021-01-01 00:56:00,12 F,9 F,88 %,NNE,6 mph,0 mph,23.78 in,0.0 in,Cloudy
1,2021-01-01 01:56:00,13 F,10 F,88 %,NNE,5 mph,0 mph,23.78 in,0.0 in,Cloudy
2,2021-01-01 02:56:00,13 F,10 F,88 %,N,6 mph,0 mph,23.79 in,0.0 in,Light Snow
3,2021-01-01 03:11:00,12 F,10 F,92 %,N,7 mph,0 mph,23.78 in,0.0 in,Light Snow
4,2021-01-01 03:56:00,14 F,10 F,84 %,N,6 mph,0 mph,23.77 in,0.0 in,Light Snow
...,...,...,...,...,...,...,...,...,...,...
549,2021-01-12 15:56:00,18 F,10 F,71 %,N,7 mph,0 mph,23.80 in,0.0 in,Mostly Cloudy
550,2021-01-12 16:56:00,20 F,14 F,78 %,E,5 mph,0 mph,23.80 in,0.0 in,Cloudy
551,2021-01-12 17:56:00,23 F,15 F,72 %,CALM,0 mph,0 mph,23.81 in,0.0 in,Light Snow
552,2021-01-12 18:56:00,25 F,18 F,75 %,NNE,7 mph,0 mph,23.82 in,0.0 in,Light Snow


In [175]:
import pickle
pickle.dump(df, open( "test_weather_df.p", "wb" ) )

In [287]:
df = pickle.load(open("jackson_hole_weather_df.p","rb"))

# Prepare data for uploading to mongodb

In [288]:
def convert_to_numeric(df,col):
    df[col]=df[col].str.extract(r'(\d+)')
    df[col] = pd.to_numeric(df[col])

In [289]:
def prep_for_mongo(df):
    convert_to_numeric(df,"Wind Speed")
    convert_to_numeric(df,"Dew Point")
    convert_to_numeric(df,"Humidity")
    convert_to_numeric(df,"Wind Gust")
    convert_to_numeric(df,"Pressure")
    convert_to_numeric(df,"Precip.")
    convert_to_numeric(df,"Temperature")

    df = df.rename(columns={"Precip.": "Precipitation"})
    return df

In [290]:
df = prep_for_mongo(df)

In [291]:
df.dtypes

Time             datetime64[ns]
Temperature               int64
Dew Point                 int64
Humidity                  int64
Wind                     object
Wind Speed                int64
Wind Gust                 int64
Pressure                  int64
Precipitation             int64
Condition                object
dtype: object

In [292]:
df.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precipitation,Condition
0,2021-01-01 00:56:00,12,9,88,NNE,6,0,23,0,Cloudy
1,2021-01-01 01:56:00,13,10,88,NNE,5,0,23,0,Cloudy
2,2021-01-01 02:56:00,13,10,88,N,6,0,23,0,Light Snow
3,2021-01-01 03:11:00,12,10,92,N,7,0,23,0,Light Snow
4,2021-01-01 03:56:00,14,10,84,N,6,0,23,0,Light Snow


# Now upload to mongodb

In [225]:
from pymongo import MongoClient
import urllib.parse

# init mongo connection
username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)
db = client["weather"]
mycol = db["jackson_hole"]

data_dict = df.to_dict("records")

# Insert collection
mycol.insert_many(data_dict)

# Now retrieve data from mongodb

In [375]:
pipeline = [
  {
    "$addFields": {
      "datetime": {
        "$toDate": {
          "$multiply": [
            "$counted_at",
            1000
          ]
        }
      }
    }
  },
  {
    "$addFields": {
      "counted_at": {
        "$convert": {
          "input": "$counted_at",
          "to": "double",
          "onError": None
        }
      },
      "computer_id": {
        "$convert": {
          "input": "$computer_id",
          "to": "string",
          "onError": None
        }
      }
    }
  },
  {
    "$match": {
      "arg_file": {
        "$in": [
          "jackson_hole.env"
        ]
      },
      "type": {
        "$nin": []
      },
      "computer_id": {
        "$in": [
          None,
          "",
          "00000000-0000-0000-0000-3cecef225486"
        ]
      }
    }
  },
  {
    "$addFields": {
      "datetime": {
        "$cond": {
          "if": {
            "$eq": [
              {
                "$type": "$datetime"
              },
              "date"
            ]
          },
          "then": "$datetime",
          "else": None
        }
      }
    }
  },
  {
    "$addFields": {
      "__alias_0": {
        "year": {
          "$year": "$datetime"
        },
        "month": {
          "$subtract": [
            {
              "$month": "$datetime"
            },
            1
          ]
        },
        "date": {
          "$dayOfMonth": "$datetime"
        },
        "hours": {
          "$hour": "$datetime"
        }
      }
    }
  },
  {
    "$group": {
      "_id": {
        "__alias_0": "$__alias_0"
      },
      "__alias_1": {
        "$sum": {
          "$cond": [
            {
              "$ne": [
                {
                  "$type": "$datetime"
                },
                "missing"
              ]
            },
            1,
            0
          ]
        }
      }
    }
  },
  {
    "$project": {
      "_id": 0,
      "__alias_0": "$_id.__alias_0",
      "__alias_1": 1
    }
  },
  {
    "$project": {
      "x": "$__alias_0",
      "y": "$__alias_1",
      "_id": 0
    }
  },
  {
    "$sort": {
      "x.year": 1,
      "x.month": 1,
      "x.date": 1,
      "x.hours": 1
    }
  },
  {
    "$limit": 5000
  }
]

In [376]:
# init mongo connection
username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)

In [377]:
traffic_results = client["traffic"]["traffic_data"].aggregate(pipeline)

In [378]:
traffic_results = list(traffic_results)

In [379]:
traffic_results[0]

{'x': {'year': 2021, 'month': 0, 'date': 8, 'hours': 3}, 'y': 190}

In [381]:
for i in traffic_results:
    i["x"] = datetime.datetime(year = i["x"]["year"], month = i["x"]["month"]+1, day = i["x"]["date"],hour = i["x"]["hours"])

In [382]:
traffic_df = pd.DataFrame.from_records(traffic_results)

In [383]:
traffic_df

,x,y
0,2021-01-08 03:00:00,190
1,2021-01-08 04:00:00,126
2,2021-01-08 05:00:00,68
3,2021-01-08 06:00:00,12
4,2021-01-08 07:00:00,26
...,...,...
119,2021-01-13 02:00:00,183
120,2021-01-13 03:00:00,121
121,2021-01-13 04:00:00,113
122,2021-01-13 05:00:00,82


# Now get back the weather data from the db

In [271]:
pipeline_weather = [
  {
    "$group": {
      "_id": {
        "__alias_0": "$Time",
        "__alias_1": "$Dew Point",
        "__alias_2": "$Wind Gust",
        "__alias_3": "$Wind Speed",
        "__alias_4": "$Temperature",
        "__alias_5": "$Pressure"
      }
    }
  },
  {
    "$project": {
      "_id": 0,
      "__alias_0": "$_id.__alias_0",
      "__alias_1": "$_id.__alias_1",
      "__alias_2": "$_id.__alias_2",
      "__alias_3": "$_id.__alias_3",
      "__alias_4": "$_id.__alias_4",
      "__alias_5": "$_id.__alias_5"
    }
  },
  {
    "$project": {
      "x": "$__alias_0",
      "y": "$__alias_1",
      "y_series_0": "$__alias_2",
      "y_series_1": "$__alias_3",
      "y_series_2": "$__alias_4",
      "y_series_3": "$__alias_5",
      "_id": 0
    }
  },
  {
    "$sort": {
      "x": 1,
      "y": 1,
      "y_series_0": 1,
      "y_series_1": 1,
      "y_series_2": 1,
      "y_series_3": 1
    }
  },
  {
    "$addFields": {
      "__multi_series": {
        "$objectToArray": {
          "Wind Gust": "$y_series_0",
          "Wind Speed": "$y_series_1",
          "Temperature": "$y_series_2",
          "Pressure": "$y_series_3",
          "Dew Point": "$y"
        }
      }
    }
  },
  {
    "$unwind": "$__multi_series"
  },
  {
    "$addFields": {
      "color": "$__multi_series.k",
      "y": "$__multi_series.v"
    }
  },
  {
    "$project": {
      "__multi_series": 0,
      "y_series_0": 0,
      "y_series_1": 0,
      "y_series_2": 0,
      "y_series_3": 0
    }
  },
  {
    "$limit": 50000
  }
]

In [272]:
weather_results = client["weather"]["jackson_hole"].aggregate(pipeline_weather)
list(weather_results)

[{'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 0, 'color': 'Wind Gust'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 6, 'color': 'Wind Speed'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 12, 'color': 'Temperature'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 23, 'color': 'Pressure'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 9, 'color': 'Dew Point'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 0, 'color': 'Wind Gust'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 5, 'color': 'Wind Speed'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 13, 'color': 'Temperature'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 23, 'color': 'Pressure'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 10, 'color': 'Dew Point'},
 {'x': datetime.datetime(2021, 1, 1, 2, 56), 'y': 0, 'color': 'Wind Gust'},
 {'x': datetime.datetime(2021, 1, 1, 2, 56), 'y': 6, 'color': 'Wind Speed'},
 {'x': datetime.datetime(2021, 1, 1, 2, 56), 'y': 13, 'color': 'Temperature'},

In [366]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

fig = px.line(df, x="Time", y=df[["Temperature","Dew Point","Wind Speed","Wind Gust","Pressure","Precipitation"]].columns)
fig 
fig.show()


traffic_df

,x,y
0,2021-01-08 03:02:00,1
1,2021-01-08 03:03:00,2
2,2021-01-08 03:04:00,1
3,2021-01-08 03:05:00,2
4,2021-01-08 03:06:00,5
...,...,...
4995,2021-01-12 18:06:00,4
4996,2021-01-12 18:07:00,8
4997,2021-01-12 18:08:00,6
4998,2021-01-12 18:09:00,7


In [385]:

fig = px.line(x = traffic_df["x"], y = traffic_df["y"])
fig 
fig.show()

In [384]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Temperature"],
                        mode='lines',
                        name='Temperature'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Wind Gust"],
                        mode='lines',
                        name='Wind Gust'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Wind Gust"],
                        mode='lines',
                        name='Wind Gust'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x = traffic_df["x"], y = traffic_df["y"],
                        mode='lines',
                        name='Traffic volume'),    

    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

In [391]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

fig = go.Figure()


# Add traces
fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Temperature"],
                        mode='lines',
                        name='Temperature',
               yaxis="y2"
              )
)


fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Condition"],
                        mode='lines',
                        name='Condition',
                        yaxis="y3")
)

fig.add_trace(
    go.Scatter(x = traffic_df["x"], y = traffic_df["y"],
                        mode='lines',
                        name='Traffic volume'),
)


# Create axis objects
fig.update_layout(
    yaxis=dict(
        title="yaxis title",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="yaxis2 title",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.15
    ),
    yaxis3=dict(
        title="yaxis3 title",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="yaxis4 title",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.85
    )
)

# Update layout properties
fig.update_layout(
    title_text="multiple y-axes example",
)
fig.show()

In [386]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[1, 2, 3],
    y=[4, 5, 6],
    name="yaxis1 data"
))


fig.add_trace(go.Scatter(
    x=[2, 3, 4],
    y=[40, 50, 60],
    name="yaxis2 data",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=[4, 5, 6],
    y=[40000, 50000, 60000],
    name="yaxis3 data",
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=[5, 6, 7],
    y=[400000, 500000, 600000],
    name="yaxis4 data",
    yaxis="y4"
))


# Create axis objects
fig.update_layout(
    xaxis=dict(
        domain=[0.3, 0.7]
    ),
    yaxis=dict(
        title="yaxis title",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="yaxis2 title",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.15
    ),
    yaxis3=dict(
        title="yaxis3 title",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="yaxis4 title",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.85
    )
)

# Update layout properties
fig.update_layout(
    title_text="multiple y-axes example",
    width=800,
)

fig.show()